<a href="https://colab.research.google.com/github/ash-rulz/TextMining/blob/main/TextMiningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG
This notebook implements a RAG pipeline for answering questions related to the first book in the Harry Potter series - "Harry Potter and the Sorcer's Stone".

The whole process can be summarized as follows:
1. **PDF splitter**: A PDF version of the book is parsed and split into smaller chunks.
2. **Sentence embedding**: These chunks of information is transformed into sentence embeddings. The sentence embedder user for this is *sentence-transformers/all-MiniLM-L6-v2*.
3. **Vector DB**: The embeddings are stored in vector DB. The vector DB used here is FAISS.
4. **Generator**: A generator based on a LLM is created. The generator used here is *google/flan-t5-large*.
5. **Retriever chain**: A retriever chain is then created. The input to a retriever chain will be a question. The question is converted into a sentence embedding which is then compared with the embedding in the vector DB. The k most similar documents are retrieved from the vector DB. These documents are passed as *context* in the custom made prompt template, along with the original question. This prompt is passed to the Generator to get the answer to the question.
6. **Evaluation**: The whole RAG pipeline is evaluated using the RAGAS framework.
7. **Improvements**: We try out different ways to improve the evaluation scores.

In [2]:
!pip install -q -U langchain pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00


In [3]:
#Load the pdf to memory
from langchain.document_loaders import PyPDFLoader
pdfLoader = PyPDFLoader("Book.pdf")
documents = pdfLoader.load()

In [4]:
#Split the file to chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=['\n\n', '\n', '(?=>\. )', ' ', ''])
docs = text_splitter.split_documents(documents)
len(docs)

1189

In [7]:
!pip install -U -q sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00


In [8]:
#Store the documents in a vector store
from langchain.embeddings import HuggingFaceEmbeddings

model_path = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

#Create vector store
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [10]:
#Example of documents retrieved from the vector DB for a question
question = "What is the name of Filch's cat?"
searchDocs = db.similarity_search_with_score(question)
print(searchDocs[0])

(Document(page_content="106Filch owned a cat called Mrs. Norris, a scrawny, dust-colored creature\nwith bulging, lamp like eyes just like Filch's. She patrolled thecorridors alone. Break a rule in front of her, put just one toe out ofline, and she'd whisk off for Filch, who'd appear, wheezing, two secondslater. Filch knew the secret passageways of the school better thananyone (except perhaps the Weasley twins) and could pop up as suddenly\nas any of the ghosts. The students all hated him, and it was the dearest", metadata={'source': 'Book.pdf', 'page': 106}), 0.7470939)


In [11]:
#Create a generator
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,pipeline
from langchain import HuggingFacePipeline

model_name_flan = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name_flan)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_flan)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer,max_new_tokens=200)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0, "max_length": 1000000},
)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [15]:
#Evidence of hallucination by the T5 model
llm_result = llm.invoke(question)
llm_result

'sam'

In [16]:
#Create a retriever
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

base_retriever = db.as_retriever(search_kwargs={"k" : 3})
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [18]:
#Evidence of how the RAG improved the result
result = retrieval_augmented_qa_chain.invoke({"question" : question})
result['response']

'Mrs. Norris'

# Evaluation

In [24]:
!pip install -q datasets tqdm

In [23]:
#Get the ground truth data from QAEval
from datasets import Dataset
#eval_dataset = Dataset.from_csv("QAEval.csv", encoding='latin1', sep = ';')
eval_dataset = Dataset.from_csv("QAEval.csv")
eval_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'ground_truth'],
    num_rows: 141
})

In [25]:
#Format the data into the RAGAS structure
from tqdm import tqdm
import pandas as pd

def create_ragas_dataset(rag_chain, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_chain.invoke({"question" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["response"],
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

basic_qa_ragas_dataset = create_ragas_dataset(retrieval_augmented_qa_chain, eval_dataset)
basic_qa_ragas_dataset

100%|██████████| 141/141 [20:55<00:00,  8.91s/it]


Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 141
})

In [26]:
# Save the dataset to a Parquet file
save_path = '/content/basic_qa_ragas_dataset.parquet'
basic_qa_ragas_dataset.to_pandas().to_parquet(save_path)

In [ ]:
#Download the file